## Prompt Chaining 結合外部工具包括 Google 搜尋、計算機和股價API


In [50]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [51]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [52]:
import requests
import json
from pprint import pp

In [53]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, format_type="text"):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens, "response_format": { "type": format_type } }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## case1: Google search

OpenAI gpt-3.5-turbo 模型的 cut-off date 是 2021/9，最新 GPT-4-0125-preview 是 2023/12。因此沒有最新資料。

透過 google search 下來的數據，結合用戶問題，減少AI幻覺的可能性 

In [54]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python -q
# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)

In [55]:
from googlesearch import search

In [56]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

In [57]:
user_query = "請問 2024 年 Nvidia 的股價表現如何?"

prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)



'重要關鍵字: 2024年, Nvidia, 股價表現'


💡 AI 回覆有時候會多 label，例如上面的 "重要關鍵字"
這時候可以改一下 prompt 的結尾，加上引導文字，讓 AI 繼續接龍下去。

## 加上引導文字

In [58]:
prompt = f"""
你是 Google 搜尋引擎，請根據以下用戶問題，擷取出和問題相關的重要關鍵字:
用戶問題: ```{user_query}```
關鍵字:
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
keywords = response
pp(response)

'2024年、Nvidia、股價、表現。'


In [59]:
search_result = google_search(keywords)
pp(search_result)

('Title: NVIDIA股價為何上漲？2024輝達股價還會漲嗎？\n'
 ' Description: 2024年2月8日 — 到2023 年12 月，Nvidia '
 '股價上漲了230%。輝達被認為是當年表現最佳的標準普爾股票之一，其成長速度令其第二大競爭對手(Meta) 相形見絀\xa0...\n'
 '\n'
 'Title: Nvidia股價表現驚人\n'
 ' Description: 2024年2月24日 — '
 '2024年到目前為止，Nvidia的股價一直持續上升，只有在2月21日時出現極短暫的下跌。自年初以來Nvidia股價已飆升近40%，在過去12 '
 '個月裡甚至飆升了225\xa0...\n'
 '\n'
 'Title: Nvidia 輝達（NVDA）財報分析2024 Q4\n'
 ' Description: 2024年2月22日 — ... 股價。 然而由於高成長不會一直持續下去，隨著業績成長放緩，公司股價也難有過去翻倍的表現。 '
 '亮眼的財報數據和指引，短期能為市場帶來信心。 但如果股價\xa0...\n'
 '\n'
 'Title: 美股史上最快NVIDIA只花8個月達標2兆美元市值\n'
 ' Description: 2024年2月25日 — 受惠AI趨勢，2023年NVIDIA股價累計大漲238.87%，2024年至今續漲59.16 '
 '... 股價表現勢必不俗。 (圖片來源：NVIDIA). ＊編者按：本文僅供參考之用，並不\xa0...\n'
 '\n'
 'Title: NVIDIA股價展望：[2024年上半年]業績、股利資訊與公平價格\n'
 ' Description: 6 天前 — NVIDIA '
 '股價展望：提供投資策略的重要訊息，包括績效分析、股利和預期股價。提供考慮公司業績的適當股票價格資訊。更深入了解NVIDIA 股票的長期價值並\xa0'
 '...\n'
 '\n'
 'Title: 2024年首選晶片股？TD Cowen分析師：Nvidia能再漲50%\n'
 ' Description: 2023年12月12日 — ... 2024年首選晶片股是輝達（Nvidia）。路透社. TD ... '
 '技嘉、緯創受惠超微AI 晶片亮相法人看好後市表現. 相關新聞. AI助輝達

In [60]:
user_query

'請問 2024 年 Nvidia 的股價表現如何?'

In [61]:
prompt = f"""

請基於以下最新事實回答:

{search_result}

用戶問題: {user_query}
"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages)
print(response)

根據最新的資訊，2024年Nvidia的股價一直呈現持續上升的趨勢，除了在2月21日出現極短暫的下跌外，股價一直在飆升。自年初以來，Nvidia的股價已經飆升近40%，在過去12個月中更是飆升了225%。這顯示Nvidia在2024年的表現非常驚人，投資者對其前景持樂觀態度。


## case2: 外部計算機

(通常為兩個 prompt 組合: 抽取 -> 運算 -> 總結) </br>
生成式 AI 本質上屬於機率模型，不具備數學運算功能，必須借助外部工具才能達成任務

STEP1: 參數抽取

In [62]:
question = "請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234"

# prompt 參考自 langchain LLMMathChain
prompt = f"""
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
Expression: ${{single line mathematical expression that solves the problem}}

Begin.

Question: What is 37593 * 67?
Expression: 37593 * 67

Question: 37593^(1/5)
Expression: 37593**(1/5)

Question: {question}
Expression:

"""

In [63]:
pp(prompt)

('\n'
 'Translate a math problem into a expression that can be executed using '
 "Python's numexpr library. Use the output of running this code to answer the "
 'question.\n'
 '\n'
 'Question: ${Question with math problem.}\n'
 'Expression: ${single line mathematical expression that solves the problem}\n'
 '\n'
 'Begin.\n'
 '\n'
 'Question: What is 37593 * 67?\n'
 'Expression: 37593 * 67\n'
 '\n'
 'Question: 37593^(1/5)\n'
 'Expression: 37593**(1/5)\n'
 '\n'
 'Question: 請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234\n'
 'Expression:\n'
 '\n')


In [64]:
messages = [{"role": "user", "content": prompt }]

response = get_completion(messages, model="gpt-4")
pp(response)

'(64 * 2 - 8) / 100 ** 1.234'


STEP2: 運算

In [65]:
import numexpr

In [66]:
answer = numexpr.evaluate(response)

str(answer)

'0.40848982764120106'

STEP3: 總結

執行工具後的回傳，你可以選擇

1. 直接回給用戶 或是 轉成某種固定格式回給客戶，就不用再一次呼叫 LLM 了
3. 再丟給 LLM 變成自然語言再回給客戶


In [67]:
question

'請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234'

In [68]:
question

'請計算 64 乘以 2 再扣掉 8，以上結果再除100後，再指數 1.234'

In [69]:
prompt = f"""
這是用戶問題: {question}

這其中的數學問題: {response} 和答案: {answer}

%%%
請用 JSON 格式回傳，遵照以下格式:
"answer": "string" // 答案

"""

messages = [{"role": "user", "content": prompt }]

response = get_completion(messages, format_type="json_object")
pp(response)



'{\n    "answer": "0.40848982764120106"\n}'


In [70]:
eval(response)["answer"]

'0.40848982764120106'

## 案例三: 串接股價 API

這次我們要求 json 格式

### Step 1: 從用戶問題中，用 prompt1 來提取出 外部工具的參數

In [71]:
query = "請問113年的3月8號股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object" )
print(result)

data = json.loads(result)



{
  "error": "請提供日期和股票代號"
}


In [101]:
query = "請問 113年的3月7號的台積電2330，股價表現如何?"

messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

1. 無需回答問題，請從用戶問題中，擷取出日期和台灣股票代號
2. 如果日期是民國年份，請加上 1911 轉成西元年份
3. 如果用戶沒有提供日期和公司名稱或股票代號，請回傳 {{ "error": "string" // 錯誤訊息 }}
4. 請回覆 JSON 格式，例如
{{
  "date": "20231015", // yyyymmdd
  "stock_code": "0050" // 台灣的股票代號
}}"""
}]

result = get_completion(messages, model="gpt-3.5-turbo-1106", format_type="json_object", temperature=1)
print(result)

data = json.loads(result)

{
  "date": "20240307",
  "stock_code": "2330"
}


### Step 2: 執行工具，拿到結果

API 參考自 https://medium.com/%E5%B7%A5%E7%A8%8B%E9%9A%A8%E5%AF%AB%E7%AD%86%E8%A8%98/5%E7%A8%AE%E6%8A%93%E5%8F%96%E5%8F%B0%E8%82%A1%E6%AD%B7%E5%8F%B2%E8%82%A1%E5%83%B9%E7%9A%84%E6%96%B9%E6%B3%95-766bf2ed9d6

In [102]:
date = data["date"]
stock_code = data["stock_code"]

html = requests.get(f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock_code}')
context = json.loads(html.text)
context

{'stat': 'OK',
 'date': '20240307',
 'title': '113年03月 2330 台積電           各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['113/03/01',
   '24,167,721',
   '16,699,995,060',
   '697.00',
   '697.00',
   '688.00',
   '689.00',
   '-1.00',
   '26,282'],
  ['113/03/04',
   '97,210,112',
   '69,868,348,694',
   '714.00',
   '725.00',
   '711.00',
   '725.00',
   '+36.00',
   '125,799'],
  ['113/03/05',
   '73,299,411',
   '53,751,887,376',
   '735.00',
   '738.00',
   '728.00',
   '730.00',
   '+5.00',
   '69,851'],
  ['113/03/06',
   '52,464,833',
   '38,203,868,985',
   '718.00',
   '738.00',
   '717.00',
   '735.00',
   '+5.00',
   '49,897'],
  ['113/03/07',
   '80,382,406',
   '61,221,034,146',
   '755.00',
   '769.00',
   '754.00',
   '760.00',
   '+25.00',
   '96,348']],
 'notes': ['符號說明:+/-/X表示漲/跌/不比價',
  '當日統計資訊含一般、零股、盤後定價、鉅額交易，不含拍賣、標購。',
  'ETF證券代號第六碼為K、M、S、C者，表示該ETF以外幣交易。',
  '權證證券代號可重複使用，權證顯示之名稱係目前存續權證之簡稱。'],
 'total': 5}

In [103]:
import pandas as pd

df = pd.DataFrame(context["data"], columns=context["fields"])
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/03/01,"24,167,721","16,699,995,060",697.00,697.00,688.00,689.00,-1.00,"26,282"
1,113/03/04,"97,210,112","69,868,348,694",714.00,725.00,711.00,725.00,+36.00,"125,799"
2,113/03/05,"73,299,411","53,751,887,376",735.00,738.00,728.00,730.00,+5.00,"69,851"
3,113/03/06,"52,464,833","38,203,868,985",718.00,738.00,717.00,735.00,+5.00,"49,897"
4,113/03/07,"80,382,406","61,221,034,146",755.00,769.00,754.00,760.00,+25.00,"96,348"


### Step 3: 用 (prompt2 + 結果) 轉成自然語言回給用戶

In [104]:
messages = [
    {"role": "user", "content": f"""
用戶問題: {query}

context: {context}
"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

根據資料顯示，113年3月7號的台積電2330股價表現如下：
- 開盤價：755.00
- 最高價：769.00
- 最低價：754.00
- 收盤價：760.00
- 漲跌價差：+25.00

可以看出當天台積電2330股價有上漲，收盤價為760.00。
